In [2]:
// declare our variables up here
let prompt, files, issue, i, examples;
const OPENAI_API_URL = 'https://api.openai.com/v1/engines/code-davinci-001/completions';


undefined

In [3]:
issue = `@file1:/path/to/file1
@file2:/path/to/file2

The amount of CPU in @file1 needs to be increased to 512M
The PVC storage amount in @file2 should be decreased to 10Gi
`;

'@file1:/path/to/file1\n' +
  '@file2:/path/to/file2\n' +
  '\n' +
  'The amount of CPU in @file1 needs to be increased to 512M\n' +
  'The PVC storage amount in @file2 should be decreased to 10Gi\n'

## Reading File Data
We'll use an `examples` directory to describe our scenarios. Every scenario consists of a directory called `files` containing the initial files that we'll be referencing, and a `issues` directory consisting of subdirectories named by the issue numbers, e.g. `issues/1`, `issues/2`, etc. Under each subdirectory, there is an `issue.md` file describing the changes to be made, and a `completions` directory where the files will be written to after being updated.


For example:

```
files/
	README.md
	index.js
	package.json
	package-lock.json
issues/
	1/
		issue.md
		completions/
			README.md
			index.js
			package.json
			package-lock.json
	2/
		issue.md
		completions/
			README.md
			index.js
			package.json
			package-lock.json
	3/
		issue.md
		completions/
			README.md
			index.js
			package.json
			package-lock.json
```



In [4]:
// import libraries
const path = require('path');
const fs = require('fs');

undefined

In [5]:
// list out the contents of the current directory
let localFiles = fs.readdirSync('./examples');

undefined

In [6]:
// return a map of {fileName => {path: path, content: content}}
const getFilesFromIssue = (issue) => {
	// extract a map of the files from the issue based on the following regex:
	// /`(@([a-zA-Z0-9_\-]+):(.+))`/g
	const fileRegex = /`(@([a-zA-Z0-9_\-]+):(.+))`/g;
	// create a map from the filename to the filepath
	const fileMap = new Map();
	// extract the string from group 3 of the regex
	let match;
	while (match = fileRegex.exec(issue)) {
		let [name, path] = [match[2], match[3]];
		if (!fileMap.has(name)) {
			// define the file object here 
			fileMap.set(name, {
				path: path,
				content: '',
				updatedContent: '',
			});
		} else {
			console.error(`duplicate file name ${name}`);
		}
	}
	return fileMap;
}


undefined

In [7]:
const populateFiles = (fileMap, rootDir) => {
	// look through the fileMap and read the contents of the file at the given path
	for (const [_, file] of fileMap) {
		let searchPath = path.join(rootDir, file.path);
		file.content = fs.readFileSync(searchPath, 'utf8');
	}
	return fileMap;
}

undefined

In [27]:
examples = new Map();
// loop through local files in ./examples
// examples is a directory 
for (const dirname of fs.readdirSync('./examples')) {
	console.log(dirname);
	examples.set(dirname, {
		issues: [],	// list of issues, sorted by their number
		files: new Map(),	// map of {fileName => {path: path, content: content, updatedContent}}
	})
	
	// issues will be a list of issue objects of the format {issue: string, issueNumber: number}
	let issues = [];

	// go through the `issues` directory and extract the `issue.md` file from each subdirectory 
	// and number the issue based on its parent directory number
	for (const issueDirname of fs.readdirSync(path.join('./examples', dirname, 'issues'))) {
		console.log('going through', issueDirname);
		let issueNumber = parseInt(issueDirname);
		let issuePath = path.join('./examples', dirname, 'issues', issueDirname, 'issue.md');
		let issue = fs.readFileSync(issuePath, 'utf8');
		issues.push({
			content: issue,
			issueNumber: issueNumber,
		});
	}
	// sort the issues by their issue number
	issues.sort((a, b) => a.issueNumber - b.issueNumber);
	examples.get(dirname).issues = issues;
	console.log("first item in issues", issues[0]);
	// populate the files map
	examples.get(dirname).files = getFilesFromIssue(issues[0].content);
	populateFiles(examples.get(dirname).files, path.join('./examples', dirname, 'files'));
}
console.log(examples);

simple-prometheus-update
going through 1
going through 2
first item in issues {
  content: '`@file1:grafana/base/datasource.yaml`\n' +
    '`@file2:grafana/base/grafana-route.yaml`\n' +
    '\n' +
    'The amount of CPU in @file1 needs to be increased to 512M\n' +
    'The PVC storage amount in @file2 should be decreased to 10Gi',
  issueNumber: 1
}
Map(1) {
  'simple-prometheus-update' => {
    issues: [ [Object], [Object] ],
    files: Map(2) { 'file1' => [Object], 'file2' => [Object] }
  }
}


undefined

In [9]:
files = new Map([
  ['file1', {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  }],
  ['file2', {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: route.openshift.io/v1\n' +
      'metadata:\n' +
      '  name: grafana\n' +
      '  annotations:\n' +
      '    kubernetes.io/tls-acme: "true"\n' +
      'spec:\n' +
      '  host: grafana.operate-first.cloud\n' +
      '  to:\n' +
      '    kind: Service\n' +
      '    name: grafana-service\n' +
      '  port:\n' +
      '    targetPort: 3000\n',
    updatedContent: ''
  }]
]);

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: ro

## Building the prompt

In [10]:
// @issue: string
// @files: map of {fileName: string => {path: string, content: string, updatedContent: string}}
const buildPrompt = (issue, files) => {
	let prompt = `# Listed below are:
# 1. Explanation of how two files need to be changed
# 2. The original files, each one separated by a '---' string
# 3. Files updated with the described changes, with a '---' string in-between each file
# 4. A '####' string, indicating the end of the document


## Description of issues:
${issue}\n

## Original files:
`;

	i = 0;
	for (const [fileName, file] of files) {
		prompt += `# @${fileName}\n${file.content}\n`;
		// only place the delimiting string if in-between files
		if (files.size > 1 && i < files.size - 1) {
			prompt += '---\n';
		}
		i++;
	}

	prompt += `
## Updated files:
`;
	return prompt;
}

undefined

In [28]:
// iterate through each example and build the corresponding prompt
for (const [dirname, example] of examples) {
	// build the initialprompt
	let initialPrompt = buildPrompt(example.issues[0].content, example.files);
	// write the prompt to a file
	fs.writeFileSync(path.join('./examples', dirname, 'prompt.md'), initialPrompt);
}

undefined

# Updating Files

We retrieve a completion from OpenAI's completion endpoint and split the files up by a '---' delimiter,
then we'll match them to their corresponding files.

Let's define a few functions to help us with this. We'll bring in the `axios` package to make our HTTP requests.

In [12]:
var axios = require('axios');

// process the completion & place it into the files' updatedContent field
const completionToFiles = (completions, files) => {
	// go through the list of completions, extract the filename and set the updated content
	for (const completion of completions) {
		// extract the file tag from the completion
		const fileTagRegex = /# \@(.+)/g;
		const match = fileTagRegex.exec(completion);
		if (match !== null) {
			const fileTag = match[1];
			if (files.has(fileTag)) {
				// find the line containing the fileTag and remove all lines up to and including the fileTag 
				const lines = completion.split('\n');
				let i = 0;
				for (const line of lines) {
					i++;
					if (line.includes(fileTag)) {
						break;
					}
				}
				// remove the lines from the completion
				const newCompletion = lines.slice(i).join('\n');
				// set the updated content
				files.get(fileTag).updatedContent = newCompletion;
			};
		}
	}
};

// to create the completion
const getCompletion = async (prompt, maxTokens, stopSequences) => {
	stopSequences = stopSequences || ['####',];
	const headers = {
		// get OPENAI_API_KEY from env
		"Authorization": `Bearer ${process.env.OPENAI_API_KEY}`,
		"Content-Type": "application/json",
	};
	// console.log("headers", headers);
	const body = {
		prompt: prompt,
		max_tokens: maxTokens | 128,
		stop: stopSequences,
		temperature: 0.12,
		top_p: 1,
		frequency_penalty: 0,
		presence_penalty: 0,
	};
	let completionFiles;

	// request the openai api using axios
	await axios.post(OPENAI_API_URL, body, { headers }).then(async (response) => {
		// update the object with the competion result
		if (response.status == 200 && response.data.choices) {
			if (response.data.choices.length > 0) {
				let completion = response.data.choices[0].text;
				// split completion into files 
				completionFiles = completion.split('---');
			} else {
				console.error("no completion found");
			}
		}
	});
	return completionFiles;
};



undefined

In [13]:
const updateFilesFromCompletion = async (files, prompt, maxTokens) => {
	const completion = await getCompletion(prompt, maxTokens, ['####',]);
	completionToFiles(completion, files);
}

undefined

In [29]:
let generateCompletionsOnExample;

undefined

In [37]:
generateCompletionsOnExample = async (basePath, example) => {
	for (let k = 0; k < example.issues.length; k++) {
		// if this is not the first issue, then we need to update the files object so that the 
		// base content property is set to the updatedContent, and the updatedContent is set to blank
		if (k > 0) {
			for (const [fileName, file] of example.files) {
				file.content = file.updatedContent;
				file.updatedContent = '';
			}
		}

		// build the initial prompt
		let initialPrompt = buildPrompt(example.issues[k].content, example.files);
		// update the files with the completion
		await updateFilesFromCompletion(example.files, initialPrompt, 512);
		
		// write the updated files into the completions directory using their same path as the original files
		for (const [fileName, file] of example.files) {
			let outputPath = path.join(basePath, 'issues', example.issues[k].issueNumber.toString(), 'completions', file.path);
			console.log('writing to: ', outputPath);
			// write the file and create parent directories, if needed
			fs.writeFileSync(outputPath, file.updatedContent);
		}
	}
}

[AsyncFunction: generateCompletionsOnExample]

Now we'll run the file updater

In [38]:
$$.async();

{
	// iterate through the issues and get a completion for each one
	for (const [dirname, example] of examples) {
		generateCompletionsOnExample(path.join('./examples', dirname), example);
	}

	prompt = buildPrompt(issue, files);
	updateFilesFromCompletion(files, prompt, 512);
}

writing to:  examples/simple-prometheus-update/issues/1/completions/grafana/base/datasource.yaml


Error: ENOENT: no such file or directory, open 'examples/simple-prometheus-update/issues/1/completions/grafana/base/datasource.yaml'
    at Object.openSync (node:fs:585:3)
    at Object.writeFileSync (node:fs:2153:35)
    at generateCompletionsOnExample (evalmachine.<anonymous>:21:7)
    at processTicksAndRejections (node:internal/process/task_queues:96:5)

undefined

In [35]:
files

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: data

In [39]:
fs.writeFileSync(path.join('./examples', 'second-dir', 'sub-dir', 'another-dir', 'prompt.md'), 'initialPrompt');
// write the file to the /examples/test/dir directory and create parent directories, if needed 

let testDir = './examples/test-dir/another-dir/tc-top.txt';
fs.mkdir(getDirName(testDir), { recursive: true}, function (err) {
	if (err) return cb(err);

	fs.writeFile(path, contents, cb);
});


Error: ENOENT: no such file or directory, open 'examples/second-dir/sub-dir/another-dir/prompt.md'